In [4]:
import pandas as pd

# combine all the data/usda_* into one dataframe
usda = pd.concat([pd.read_csv(f'data/usda_data_{year}.csv') for year in range(2019, 2021)])

# combine all the data/weather_* into one dataframe
weather = pd.concat([pd.read_csv(f'data/weather_data_{year}.csv') for year in range(2019, 2021)])


In [5]:
# GDD calculation function
def calculate_gdd(df, base_temp=50, upper_temp=86):
    """
    Calculate Growing Degree Days (GDD) for corn.
    """
    df['TMAX'] = df['TMAX'].clip(lower=base_temp, upper=upper_temp)
    df['TMIN'] = df['TMIN'].clip(lower=base_temp)
    df['TAVG'] = (df['TMAX'] + df['TMIN']) / 2
    df['GDD'] = df['TAVG'] - base_temp
    return df

In [6]:
weather = calculate_gdd(weather)